<h2 style="color:MediumVioletRed; font-family:Cursive"><b>Goal of creating this Notebook🎯</h2>

1. Perform Clustering / Segmentation on the dataset and identify popular customer groups along with their definitions/rules
2. Perform Location-wise analysis to identify regional trends in India
3. Perform transaction-related analysis to identify interesting trends that can be used by a bank to improve / optimi their user experiences
4. Customer Recency, Frequency, Monetary analysis
5. Network analysis or Graph analysis of customer data.

**Table of contents of this notebook: 📭**

1. [Importing Necessary Libraries](#1)

2. [Data Collection](#2)

3. [Data Cleaning](#3)

4. [Exploratory Data Analysis](#4)

<h2  style="color:darkmagenta;text-align: center; background-color: AliceBlue;padding: 20px;">1. Importing Libraries</h2><a id = "1"></a>

In [1]:
import re
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("fivethirtyeight")
import seaborn as sns

<h2  style="color:darkmagenta;text-align: center; background-color: AliceBlue;padding: 20px;">2. Data Collection</h2><a id = "2"></a>

In [2]:
df = pd.read_csv("../../data/bank_transactions.csv")
df.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


In [3]:
df = df.rename(columns={'TransactionAmount (INR)':'TransactionAmount'})

In [4]:
initialRows = df.shape[0]
df.shape

(1048567, 9)

<h2  style="color:darkmagenta;text-align: center; background-color: AliceBlue;padding: 20px;">3. Data Cleaning</h2><a id = "3"></a>

In [ ]:
df.info()

<h3>The amount of null data and duplicated is calculated</h3>

In [ ]:
def check(df):
    l=[]
    columns=df.columns
    for col in columns:
        dtypes=df[col].dtypes
        nunique=df[col].nunique()
        sum_null=df[col].isnull().sum()
        l.append([col,dtypes,nunique,sum_null])
    df_check=pd.DataFrame(l)
    df_check.columns=['column','dtypes','nunique','sum_null']
    return df_check 
check(df)

In [5]:
df.dropna(inplace=True)

In [ ]:
df.duplicated().sum()

<h3>The CustomerDOB column is analyzed because it may contain atypical data.</h3>

In [ ]:
df['CustomerDOB'].value_counts()

<p>Data 1/1/1800 are deleted because it is not possible to define whether they are children, adults or persons without date of birth.</p>

In [6]:
df = df.loc[~(df['CustomerDOB'] == '1/1/1800')]
df['CustomerDOB'].value_counts()

1/1/89      809
1/1/90      784
6/8/91      698
1/1/91      665
1/1/92      631
           ... 
23/2/05       1
28/11/42      1
23/9/49       1
14/3/40       1
24/10/44      1
Name: CustomerDOB, Length: 17232, dtype: int64

In [7]:
print("min: " + df['CustomerDOB'].min() + " max: " + df['CustomerDOB'].max())

min: 1/1/00 max: 9/9/97


<p>Convert type of columns TransactionDate,CustomerDOB from string to datetime.</p>

In [8]:
df['CustomerDOB'] = pd.to_datetime(df['CustomerDOB'], dayfirst=True)
df[df['CustomerDOB'] > '2001-01-01']

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount
1,T2,C2142763,2057-04-04,M,JHAJJAR,2270.69,2/8/16,141858,27999.00
5,T6,C1536588,2072-10-08,F,ITANAGAR,53609.20,2/8/16,173940,676.00
15,T16,C8334633,2068-07-10,F,NEW DELHI,1283.12,1/8/16,125725,250.00
30,T31,C7648712,2067-01-21,F,NEW DELHI,298461.05,3/8/16,160642,500.00
51,T52,C6637047,2066-07-25,M,PUNE,9956.49,5/8/16,133141,145.00
...,...,...,...,...,...,...,...,...,...
1048506,T1048507,C3333984,2070-09-20,M,NAVI MUMBAI,41.82,18/9/16,182310,878.18
1048510,T1048511,C3429427,2048-07-15,M,HOWRAH,416244.74,18/9/16,184959,1695.00
1048536,T1048537,C2939947,2033-10-05,M,NEW DELHI,92173.35,18/9/16,184556,1319.04
1048544,T1048545,C3712582,2071-10-05,M,MUMBAI,25414.52,18/9/16,182015,400.00


In [ ]:
print(df['CustomerDOB'].min(), df['CustomerDOB'].max())

In [9]:
df.loc[df['CustomerDOB'].dt.year > 2000, 'CustomerDOB'] -= pd.DateOffset(years=100)
df[df['CustomerDOB'] > '2001-01-01']

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount
3696,T3697,C1934386,2002-12-09,M,NEW DELHI,51469.13,21/10/16,171357,963.0
5920,T5921,C7424249,2001-03-04,M,BANGALORE,4550.32,23/9/16,133547,1197.4
6714,T6715,C7828452,2002-08-20,M,AMRITSAR,2458.12,26/9/16,232151,50.0
9674,T9675,C7937849,2003-12-08,M,BAREILLY,2593.90,25/9/16,193751,3294.0
19811,T19812,C7421182,2002-02-07,M,MOHALI,97442.06,26/9/16,115008,0.0
...,...,...,...,...,...,...,...,...,...
1040958,T1040959,C3074853,2002-03-31,F,MUKTSAR,14552.60,18/9/16,144908,101.0
1044338,T1044339,C6822618,2001-07-11,F,PATNA,92052.93,18/9/16,103104,5884.0
1045058,T1045059,C3422655,2001-07-11,F,PATNA,92052.93,18/9/16,80219,368.0
1045119,T1045120,C8943140,2002-07-17,F,AGRA,9315.13,18/9/16,3547,600.0


In [10]:
print(df['CustomerDOB'].min(), df['CustomerDOB'].max())

1904-01-20 00:00:00 2003-12-21 00:00:00


In [11]:
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'])

In [ ]:
print(df['TransactionDate'].min(), df['TransactionDate'].max())

<p> calculate customer age :</p>
<p> CustomerDOB is the birth date of the customer , TransactionDate : is the date of transaction that customer is done
</p>

In [12]:
df['CustomerAge'] = df['TransactionDate'].dt.year - df['CustomerDOB'].dt.year

In [13]:
print("min: " + str(df['CustomerAge'].min()) + " max: " + str(df['CustomerAge'].max()))

min: 13 max: 112


In [ ]:
df.CustGender.value_counts()

In [ ]:
num_col = df.select_dtypes(include=np.number)
cat_col = df.select_dtypes(exclude=np.number)

In [ ]:
plt.style.use("fivethirtyeight")
plt.figure(figsize=(30,30))
for index,column in enumerate(num_col):
    plt.subplot(7,4,index+1)
    sns.boxplot(data=num_col,x=column)
    
plt.tight_layout(pad = 1.0)

In [ ]:
print("Number of initial rows: ", initialRows)
print("Number of rows after: ", df.shape[0])
print("Number of rows deleted: ", initialRows - df.shape[0])
print("Percentage of rows deleted: ", (initialRows - df.shape[0]) / initialRows * 100, "%")

<h3 style="color:MediumVioletRed; font-family:Cursive">Observations 💡:</h3>
<p>There is no outliers.I can not say that there is a outliers.May Some variables seem like outliers but they are not, I decided spare any data.</p>

<img src="https://miro.medium.com/max/638/0*JddVeZpHXdElEec_" alt="RFM" >
<p style="color:DarkSlateGray">those two articles will help you to understand this topic 📜:</p>
<a href="https://connectif.ai/en/what-are-rfm-scores-and-how-to-calculate-them/">What Are RFM Scores and How To Calculate Them</a>
<br>
<a href="https://www.datacamp.com/tutorial/introduction-customer-segmentation-python">Introduction to Customer Segmentation in Python</a>

<p>Recency: number of days since the last purchase or order so I will create a new column of TransactionDate to subtract the last transaction from the first transaction</p>

In [ ]:
#df['TransactionDate1'] ==> I will keep it to do visaulization
df['TransactionDate1']=df['TransactionDate'] # ==> to calculate the minimum (first transaction)
df['TransactionDate2']=df['TransactionDate'] # ==> to calculate the maximum (last transaction)

In [ ]:
#Creating MRF Table Strategy

RFM_df = df.groupby("CustomerID").agg({
                                        "TransactionID" : "count",
                                        "CustGender" : "first",
                                        "CustLocation":"first",
                                        "CustAccountBalance"  : "mean",
                                        "TransactionTime": "mean",
                                        "TransactionAmount" : "mean",
                                        "CustomerAge" : "median",
                                        "TransactionDate2":"max",
                                        "TransactionDate1":"min",
                                        "TransactionDate":"median"
                        })

RFM_df = RFM_df.reset_index()
RFM_df.head()

In [ ]:
RFM_df.shape

In [ ]:
RFM_df.drop(columns=["CustomerID"],inplace=True)

In [ ]:
RFM_df.rename(columns={"TransactionID":"Frequency"},inplace=True)

In [ ]:
RFM_df['Recency']=RFM_df['TransactionDate2']-RFM_df['TransactionDate1']
RFM_df['Recency']=RFM_df['Recency'].astype(str)

In [ ]:
# extract the day
x='18 day'
re.search('\d+',x).group()

In [ ]:
RFM_df['Recency']=RFM_df['Recency'].apply(lambda x :re.search('\d+',x).group())
RFM_df['Recency']=RFM_df['Recency'].astype(int)

<p> 0 days mean that a customer has done transaction recently one time by logic so I will convert 0 to 1 </p>

In [ ]:
def rep_0(i):
    if i==0:
        return 1
    else:
        return i
RFM_df['Recency']=RFM_df['Recency'].apply(rep_0)

In [ ]:
RFM_df.drop(columns=["TransactionDate1","TransactionDate2"],inplace=True)

In [ ]:
# to claculate the otliers for each feature
lower_list=[]
upper_list=[]
num_list=[]
perc_list=[]
cols=['Frequency', 'CustAccountBalance','TransactionAmount', 'CustomerAge', 'Recency']
for i in cols:
    Q1 = RFM_df[i].quantile(0.25)
    Q3 = RFM_df[i].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    # calculate number of outliers
    num=RFM_df[(RFM_df[i] < lower) | (RFM_df[i] > upper)].shape[0]
    # calculate percentage of outliers
    perc = (num / RFM_df.shape[0]) * 100
    lower_list.append(lower)
    upper_list.append(upper)
    num_list.append(num)
    perc_list.append(round(perc,2))

    
dic={'lower': lower_list, 'upper': upper_list, 'outliers': num_list, 'Perc%':perc_list }
outliers_df=pd.DataFrame(dic,index=['Frequency', 'CustAccountBalance','TransactionAmount', 'CustomerAge', 'Recency'])
outliers_df

<p>May Some variables seem like outliers but they are not, I decided spare any data.</p>

In [ ]:
RFM_df.head()

<h2  style="color:darkmagenta;text-align: center; background-color: AliceBlue;padding: 20px;">4. Exploratory Data Analysis</h2><a id = "4"></a>

In [ ]:
RFM_df.describe()

In [ ]:
# correlation between features
plt.figure(figsize=(7,5))
correlation=RFM_df.corr()
sns.heatmap(correlation,vmin=None,
    vmax=0.8,
    cmap='rocket_r',
    annot=True,
    fmt='.1f',
    linecolor='white',
    cbar=True);

In [ ]:
plt.style.use("fivethirtyeight")
sns.pairplot(RFM_df,hue='Frequency');

In [ ]:
plt.style.use("fivethirtyeight")
chart=sns.countplot(x='Frequency',data=RFM_df,palette='rocket', order = RFM_df['Frequency'].value_counts().index)
plt.title("Frequency",
          fontsize='20',
          backgroundcolor='AliceBlue',
          color='magenta');

In [ ]:
plt.style.use("fivethirtyeight")
fig,ax=plt.subplots(ncols=2,nrows=1,figsize=(15,5))
palette_color = sns.color_palette('rocket')
ax[0].hist(x=RFM_df['CustomerAge'],color='purple')
ax[0].set_title("Distribution of Customer Age")
ax[1].pie(RFM_df['CustGender'].value_counts(),autopct='%1.f%%',colors=palette_color,labels=['Male','Female'])
ax[1].set_title("Customer Gender")
plt.tight_layout();

In [ ]:
plt.style.use("fivethirtyeight")
plt.figure(figsize=(15,7))
chart=sns.countplot(y='CustLocation',data=RFM_df,palette='rocket', order = RFM_df['CustLocation'].value_counts()[:20].index)
plt.title("Most 20 Location of Customer ",
          fontsize='20',
          backgroundcolor='AliceBlue',
          color='magenta');

In [ ]:
plt.style.use("fivethirtyeight")
sns.scatterplot(x='TransactionAmount',y='CustAccountBalance',data=RFM_df,palette='rocket',hue='Frequency',size='Recency' )
plt.title("TransactionAmount (INR) and CustAccountBalance",
          fontsize='20',
          backgroundcolor='AliceBlue',
          color='magenta');

In [ ]:
# difference between maximum and minimum date
RFM_df['TransactionDate'].max()-RFM_df['TransactionDate'].min()

In [ ]:
RFM_df=RFM_df.sort_values(by='TransactionDate')
groupbby_month=RFM_df.groupby([pd.Grouper(key='TransactionDate',freq='M')]).mean()
print(groupbby_month.shape)
groupbby_month

In [ ]:
plt.style.use("fivethirtyeight")
fig,ax=plt.subplots(ncols=2,nrows=1,figsize=(15,5))
ax[0].plot(groupbby_month.index,groupbby_month['CustAccountBalance'],color='purple',marker='o',label='Customer Account Balance')
ax[0].set_title('Cust Account Balance Over The Time')
ax[1].plot(groupbby_month.index,groupbby_month['TransactionAmount'],color='purple',marker='o',label='Transaction Amount')
ax[1].set_title("Transaction Amount(INR) Over The Time")
plt.legend();